<a href="https://colab.research.google.com/github/av-dutta/NYC-Taxi-Trip-Time-Prediction/blob/main/NYC_Taxi_Trip_Time_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Taxi trip time Prediction : Predicting total ride duration of taxi trips in New York City</u></b>

## <b> Problem Description </b>

### Your task is to build a model that predicts the total ride duration of taxi trips in New York City. Your primary dataset is one released by the NYC Taxi and Limousine Commission, which includes pickup time, geo-coordinates, number of passengers, and several other variables.

## <b> Data Description </b>

### The dataset is based on the 2016 NYC Yellow Cab trip record data made available in Big Query on Google Cloud Platform. The data was originally published by the NYC Taxi and Limousine Commission (TLC). The data was sampled and cleaned for the purposes of this project. Based on individual trip attributes, you should predict the duration of each trip in the test set.

### <b>NYC Taxi Data.csv</b> - the training set (contains 1458644 trip records)


### Data fields
* #### id - a unique identifier for each trip
* #### vendor_id - a code indicating the provider associated with the trip record
* #### pickup_datetime - date and time when the meter was engaged
* #### dropoff_datetime - date and time when the meter was disengaged
* #### passenger_count - the number of passengers in the vehicle (driver entered value)
* #### pickup_longitude - the longitude where the meter was engaged
* #### pickup_latitude - the latitude where the meter was engaged
* #### dropoff_longitude - the longitude where the meter was disengaged
* #### dropoff_latitude - the latitude where the meter was disengaged
* #### store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
* #### trip_duration - duration of the trip in seconds

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
dataset = pd.read_csv("/content/drive/MyDrive/capstone Alma Better/Data & Resources/NYC Taxi Data.csv")
dataset.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [5]:
#Checking dataset information
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


Hence there is no null values in our dataset

In [6]:
#Converting pickup and dropoff into datetime format
dataset['pickup_datetime']= pd.to_datetime(dataset['pickup_datetime'],format = '%Y-%m-%d %H:%M:%S')
dataset['dropoff_datetime']= pd.to_datetime(dataset['dropoff_datetime'],format = '%Y-%m-%d %H:%M:%S')

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1458644 non-null  object        
 1   vendor_id           1458644 non-null  int64         
 2   pickup_datetime     1458644 non-null  datetime64[ns]
 3   dropoff_datetime    1458644 non-null  datetime64[ns]
 4   passenger_count     1458644 non-null  int64         
 5   pickup_longitude    1458644 non-null  float64       
 6   pickup_latitude     1458644 non-null  float64       
 7   dropoff_longitude   1458644 non-null  float64       
 8   dropoff_latitude    1458644 non-null  float64       
 9   store_and_fwd_flag  1458644 non-null  object        
 10  trip_duration       1458644 non-null  int64         
dtypes: datetime64[ns](2), float64(4), int64(3), object(2)
memory usage: 122.4+ MB


In [8]:
import datetime as dt
dataset['ptime'] = dataset['pickup_datetime'].dt.time
dataset['dtime'] = dataset['dropoff_datetime'].dt.time

In [9]:
dataset.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,ptime,dtime
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,17:24:55,17:32:30
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,00:43:35,00:54:38


In [10]:
dataset.shape

(1458644, 13)

In [11]:
(dataset['vendor_id'].unique())

array([2, 1])

In [12]:
dataset['store_and_fwd_flag'].value_counts()

N    1450599
Y       8045
Name: store_and_fwd_flag, dtype: int64

In [13]:
(dataset['passenger_count'].unique())

array([1, 6, 4, 2, 3, 5, 0, 7, 9, 8])

In [14]:
dataset['passenger_count'].value_counts()

1    1033540
2     210318
5      78088
3      59896
6      48333
4      28404
0         60
7          3
9          1
8          1
Name: passenger_count, dtype: int64

In [15]:
df = dataset.copy()

In [16]:
df = df[(df['passenger_count'] >0) & (df['passenger_count'] <7)]

In [17]:
df.passenger_count.unique()

array([1, 6, 4, 2, 3, 5])

In [18]:
df.drop(['id','pickup_datetime','dropoff_datetime'],axis=1,inplace=True)

In [19]:
df.shape

(1458579, 10)

In [20]:
df.head(2)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,ptime,dtime
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,455,17:24:55,17:32:30
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,663,00:43:35,00:54:38


In [21]:
df['pickup_longitude'].values

array([-73.98215485, -73.98041534, -73.97902679, ..., -73.95912933,
       -73.98207855, -73.97953796])

In [22]:
df.head(2)

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,ptime,dtime
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,455,17:24:55,17:32:30
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,663,00:43:35,00:54:38


In [25]:
from math import sin, cos, sqrt, atan2, radians

def distlatlong(lon1, lat1, lon2, lat2):
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
  # The Haversine Formula
    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = 6373.0 * c
    return distance


In [27]:
df['distance'] = [distlatlong(df.pickup_longitude[i],
                              df.pickup_latitude[i],
                              df.dropoff_longitude[i],
                              df.dropoff_latitude[i]) 
                for i in df.index]

In [28]:
df.head()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,ptime,dtime,distance
0,2,1,-73.982155,40.767937,-73.964630,40.765602,N,455,17:24:55,17:32:30,1.498991
1,1,1,-73.980415,40.738564,-73.999481,40.731152,N,663,00:43:35,00:54:38,1.806074
2,2,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,11:35:24,12:10:48,6.387103
3,2,1,-74.010040,40.719971,-74.012268,40.706718,N,429,19:32:31,19:39:40,1.485965
4,2,1,-73.973053,40.793209,-73.972923,40.782520,N,435,13:30:55,13:38:10,1.188962


In [30]:
df.drop(['vendor_id','passenger_count'],axis=1,inplace=True)

In [32]:
df['distance'] = round(df['distance'],3)

In [33]:
df.head()

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,ptime,dtime,distance
0,-73.982155,40.767937,-73.964630,40.765602,N,455,17:24:55,17:32:30,1.499
1,-73.980415,40.738564,-73.999481,40.731152,N,663,00:43:35,00:54:38,1.806
2,-73.979027,40.763939,-74.005333,40.710087,N,2124,11:35:24,12:10:48,6.387
3,-74.010040,40.719971,-74.012268,40.706718,N,429,19:32:31,19:39:40,1.486
4,-73.973053,40.793209,-73.972923,40.782520,N,435,13:30:55,13:38:10,1.189


In [ ]:
from sklearn.preprocessing import Label_encoding